In [1]:
import arcgis.network as network

In [2]:
analysis_url = my_gis.properties.helperServices.closestFacility.url
analysis_url

NameError: name 'my_gis' is not defined

In [3]:
my.gis=jensen.k_NU

NameError: name 'jensen' is not defined

In [4]:
my.gis='jensen.k_NU'

NameError: name 'my' is not defined

In [5]:
my_gis = GIS(profile="jensen.k_NU")

NameError: name 'GIS' is not defined

In [6]:
my_gis = "jensen.k_NU"

In [7]:
analysis_url = my_gis.properties.helperServices.closestFacility.url
analysis_url

AttributeError: 'str' object has no attribute 'properties'

In [8]:
from IPython.display import HTML
import pandas as pd
from arcgis.gis import GIS

#connect to your GIS
user_name = 'jensen.k_NU'
password = 'Corona1993*'
my_gis = GIS('https://www.arcgis.com', user_name, password)

In [9]:
import arcgis.network as network

In [10]:
analysis_url = my_gis.properties.helperServices.closestFacility.url
analysis_url

'https://route.arcgis.com/arcgis/rest/services/World/ClosestFacility/NAServer/ClosestFacility_World'

In [11]:
cf_layer = network.ClosestFacilityLayer(analysis_url, gis=my_gis)

In [12]:
from arcgis import geocoding

In [13]:
hospitals_addresses = ['Estrada Adhemar Bebiano, 339 Del Castilho, Rio de Janeiro RJ, 21051-370, Brazil',
                       'R. José dos Reis Engenho de Dentro, Rio de Janeiro RJ, 20750-000, Brazil',
                       'R. Dezessete, s/n Maré, Rio de Janeiro RJ, 21042-010, Brazil',
                       'Rua Dr. Miguel Vieira Ferreira, 266 Ramos, Rio de Janeiro RJ, Brazil']

In [14]:
from arcgis.features import Feature, FeatureSet

In [15]:
hosp_feat_list = []

for address in hospitals_addresses:
    hit = geocoding.geocode(address)[0]
    hosp_feat = Feature(geometry=hit['location'], attributes=hit['attributes'])

    hosp_feat_list.append(hosp_feat)

In [16]:
hospitals_fset = FeatureSet(features=hosp_feat_list, 
                            geometry_type='esriGeometryPoint', 
                            spatial_reference={'latestWkid': 4326})

In [17]:
from IPython.display import HTML

In [18]:
map1 = my_gis.map('Rio de Janeiro, Brazil')
map1

MapView(layout=Layout(height='400px', width='100%'))

In [19]:
map1.draw(hospitals_fset, symbol={"type": "esriSMS","style": "esriSMSSquare",
                                  "color": [76,115,0,255],"size": 8,})

In [20]:
incident_coords = '-43.281206,-22.865676'
reverse_geocode = geocoding.reverse_geocode({"x": incident_coords.split(',')[0], 
                                              "y": incident_coords.split(',')[1]})

incident_feature = Feature(geometry=reverse_geocode['location'], 
                           attributes=reverse_geocode['address'])

In [21]:
incident_fset = FeatureSet([incident_feature], geometry_type='esriGeometryPoint',
                          spatial_reference={'latestWkid': 4326})

In [22]:
map1.draw(incident_fset, symbol={"type": "esriSMS","style": "esriSMSCircle","size": 8})

In [23]:
result = cf_layer.solve_closest_facility(incidents=incident_fset,
                                        facilities=hospitals_fset,
                                        default_target_facility_count=4,
                                        return_facilities=True,
                                        impedance_attribute_name='TravelTime',
                                        accumulate_attribute_names=['Kilometers','TravelTime'])

In [24]:
result.keys()

dict_keys(['messages', 'routes', 'facilities'])

In [25]:
result['routes'].keys()

dict_keys(['fieldAliases', 'geometryType', 'spatialReference', 'features'])

In [26]:
result['routes']['features'][0].keys()

dict_keys(['attributes', 'geometry'])

In [27]:
line_feat_list = []
for line_dict in result['routes']['features']:
    f1 = Feature(line_dict['geometry'], line_dict['attributes'])
    line_feat_list.append(f1)

In [28]:
routes_fset = FeatureSet(line_feat_list, 
                         geometry_type=result['routes']['geometryType'],
                         spatial_reference= result['routes']['spatialReference'])

In [29]:
allocation_line_symbol_4 = {'type': 'esriSLS', 'style': 'esriSLSSolid',
                                'color': [0,0,255,100], 'width': 6}
map1.draw(routes_fset, symbol = allocation_line_symbol_4)

In [30]:
map1.clear_graphics()

In [31]:
df1 = routes_fset.sdf
df1

C:\Users\kylie\Anaconda3\envs\arcgispythonapi_real\lib\site-packages\IPython\lib\pretty.py:402: FutureWarning: 'ExtensionArray._formatting_values' is deprecated. Specify 'ExtensionArray._formatter' instead.
  return _repr_pprint(obj, self, cycle)
C:\Users\kylie\Anaconda3\envs\arcgispythonapi_real\lib\site-packages\pandas\io\formats\html.py:606: FutureWarning: 'ExtensionArray._formatting_values' is deprecated. Specify 'ExtensionArray._formatter' instead.
  super().render()


,ObjectID,FacilityID,FacilityRank,Name,IncidentCurbApproach,FacilityCurbApproach,IncidentID,StartTime,EndTime,StartTimeUTC,...,Total_Minutes,Total_TravelTime,Total_Miles,Total_Kilometers,Total_TimeAt1KPH,Total_WalkTime,Total_TruckMinutes,Total_TruckTravelTime,Shape_Length,SHAPE
0,1,1,1,Rua Tangapeme 133 - Location 1,1,1,1,None,None,None,...,None,5.225825,1.331738,2.143224,None,None,None,None,0.020191,"{""paths"": [[[-43.28127999999998, -22.865709999..."
1,2,4,2,Rua Tangapeme 133 - Location 4,1,2,1,None,None,None,...,None,11.971893,3.238469,5.211810,None,None,None,None,0.048701,"{""paths"": [[[-43.28127999999998, -22.865709999..."
2,3,3,3,Rua Tangapeme 133 - Location 3,1,2,1,None,None,None,...,None,14.426142,4.534989,7.298357,None,None,None,None,0.068872,"{""paths"": [[[-43.28127999999998, -22.865709999..."
3,4,2,4,Rua Tangapeme 133 - Location 2,1,1,1,None,None,None,...,None,18.209707,5.016254,8.072878,None,None,None,None,0.076224,"{""paths"": [[[-43.28127999999998, -22.865709999..."


In [32]:
df1['facility_address'] = hospitals_addresses
df1['incident_address'] = [incident_feature.attributes['Match_addr'] for i in range(len(hospitals_addresses))]

In [33]:
df1[['facility_address','incident_address','Total_Miles','Total_TravelTime']]

,facility_address,incident_address,Total_Miles,Total_TravelTime
0,"Estrada Adhemar Bebiano, 339 Del Castilho, Rio...","Rua Tangapeme 133, Inhaúma, Rio de Janeiro, 20...",1.331738,5.225825
1,"R. José dos Reis Engenho de Dentro, Rio de Jan...","Rua Tangapeme 133, Inhaúma, Rio de Janeiro, 20...",3.238469,11.971893
2,"R. Dezessete, s/n Maré, Rio de Janeiro RJ, 210...","Rua Tangapeme 133, Inhaúma, Rio de Janeiro, 20...",4.534989,14.426142
3,"Rua Dr. Miguel Vieira Ferreira, 266 Ramos, Rio...","Rua Tangapeme 133, Inhaúma, Rio de Janeiro, 20...",5.016254,18.209707
